In [1]:
import pyuvsim
import pyuvdata
import pyradiosky
import astropy
import numpy as np
import importlib
importlib.reload(pyuvsim)

<module 'pyuvsim' from '/opt/devel/rbyrne/envs/py310/lib/python3.10/site-packages/pyuvsim/__init__.py'>

In [2]:
uv = pyuvdata.UVData()
uv.read("/lustre/rbyrne/2025-06-13/20250613_070132_55MHz.ms")  # Choose an arbitrary data file for reference

Setting telescope_location to value in known_telescopes for OVRO-LWA.
The uvw_array does not match the expected values given the antenna positions. The largest discrepancy is 12.14008682625115 meters. This is a fairly common situation but might indicate an error in the antenna positions, the uvws or the phasing.


In [3]:
freq_hz = 50e6
channel_width = 2e3
#total_time_interval_s = 10
total_time_interval_s = 1
#total_freq_interval_hz = 200e3
total_freq_interval_hz = 4e3
integration_time = 0.1
time_array_s = np.arange(-total_time_interval_s/2, total_time_interval_s/2, integration_time)
uv.freq_array = np.arange(freq_hz-total_freq_interval_hz/2, freq_hz+total_freq_interval_hz/2, channel_width)
uv.Nfreqs = len(uv.freq_array)
uv.time_array = np.mean(uv.time_array) + time_array_s/86400
uv.Ntimes = len(time_array_s)
uv.Nbls = 1
uv.Nblts = uv.Ntimes
uv.Npols = 1
uv.Nants_data = 2
uv.Nspws = 1
uv.flex_spw_id_array = np.full(uv.Nfreqs, np.mean(uv.spw_array), dtype=int)
uv.ant_1_array = np.zeros(uv.Nblts, dtype=int)
uv.ant_2_array = np.ones(uv.Nblts, dtype=int)
uv.channel_width = np.full(uv.Nfreqs, channel_width)
uv.integration_time = np.full(uv.Nblts, integration_time)
uv.baseline_array = np.full(uv.Nblts, 2048 + 2**16)
uv.data_array = np.zeros((uv.Nblts, uv.Nfreqs, uv.Npols), dtype=complex)
uv.flag_array = np.zeros((uv.Nblts, uv.Nfreqs, uv.Npols), dtype=bool)
uv.nsample_array = np.ones((uv.Nblts, uv.Nfreqs, uv.Npols), dtype=float)
uv.polarization_array = np.array([-5])
uv.set_lsts_from_time_array()
uv.phase_center_app_ra = uv.phase_center_app_ra[:uv.Nblts]
uv.phase_center_app_dec = uv.phase_center_app_dec[:uv.Nblts]
uv.phase_center_frame_pa = uv.phase_center_frame_pa[:uv.Nblts]
uv.phase_center_id_array = uv.phase_center_id_array[:uv.Nblts]
uv.scan_number_array = None
uv.uvw_array = np.zeros((uv.Nblts, 3), dtype=float)
uv.phase_to_time(np.mean(uv.time_array))
# Redefine uvws after phasing
uv.uvw_array = np.zeros((uv.Nblts, 3), dtype=float)
uv.uvw_array[:, 0] = 3e3


In [4]:
uv.uvw_array

array([[3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.],
       [3000.,    0.,    0.]])

In [5]:
beam = pyuvdata.UVBeam()
beam.read("/lustre/rbyrne/LWA_10to100_MROsoil_efields.fits")
beam.select(feeds="e")
beam.peak_normalize()
beam_use_freq = 50e6
use_freq_ind = np.where(beam.freq_array == beam_use_freq)[0]
beam.data_array[:, :, :, :, :] = beam.data_array[:, :, use_freq_ind, :, :]  # Make the beam frequency-invariant
beam_list = pyuvsim.BeamList(beam_list=[beam])

In [6]:
zenith_loc = astropy.coordinates.SkyCoord([astropy.coordinates.ICRS(ra=np.mean(uv.lst_array)*astropy.units.rad, dec=uv.telescope.location.lat)])

In [7]:
uv.telescope.location.lat

<Latitude 37.23977727 deg>

In [8]:
90-38

52

In [9]:
ra_offset_vals_deg = np.linspace(-60, 60, num=11)
dec_offset_vals_deg = np.linspace(-52, 52, num=11)
ra_vals_expanded, dec_vals_expanded = np.meshgrid(ra_offset_vals_deg, dec_offset_vals_deg)
ra_offset_vals_flattened = ra_vals_expanded.flatten()
dec_offset_vals_flattened = dec_vals_expanded.flatten()
source_coords = []
for ind in range(len(ra_offset_vals_flattened)):
    new_coords = astropy.coordinates.SkyCoord([astropy.coordinates.ICRS(
        ra=np.mean(uv.lst_array)*astropy.units.rad + ra_offset_vals_flattened[ind]*astropy.units.deg, 
        dec=uv.telescope.location.lat + dec_offset_vals_flattened[ind]*astropy.units.deg
    )])
    source_coords.append(new_coords)


In [10]:
#for ind in range(len(ra_offset_vals_flattened)):
for ind in range(1):
    cat = pyradiosky.SkyModel(
        skycoord = source_coords[ind],
        name = ["source1"],
        spectral_type = "spectral_index",
        spectral_index = [0.],
        reference_frequency = ([np.mean(uv.freq_array)])*astropy.units.Hz,
        stokes = np.array([1,0,0,0]).reshape(4,1,1) * astropy.units.Jy,
    )
    output_uv = pyuvsim.uvsim.run_uvdata_uvsim(
        input_uv=uv,
        beam_list=beam_list,
        beam_dict=None,  # Same beam for all ants
        catalog=cat,
        quiet=False,
    )
    output_uv.write_uvfits(f"/lustre/rbyrne/decorr_sims/source{ind+1:03d}.uvfits")

The uvw_array does not match the expected values given the antenna positions. The largest discrepancy is 2963.7582907998344 meters. This is a fairly common situation but might indicate an error in the antenna positions, the uvws or the phasing.


Nbls: 1
Ntimes: 10
Nfreqs: 2
Nsrcs: 1
Tasks:  20.0
5.00% completed. 0:00:01.118556  elapsed. 0:00:21.252564 remaining. 

10.00% completed. 0:00:02.180075  elapsed. 0:00:19.620679 remaining. 

15.00% completed. 0:00:02.182081  elapsed. 0:00:12.365126 remaining. 

20.00% completed. 0:00:03.251665  elapsed. 0:00:13.006660 remaining. 

25.00% completed. 0:00:03.253954  elapsed. 0:00:09.761861 remaining. 

30.00% completed. 0:00:04.309335  elapsed. 0:00:10.055114 remaining. 

35.00% completed. 0:00:04.311233  elapsed. 0:00:08.006575 remaining. 

40.00% completed. 0:00:05.440817  elapsed. 0:00:08.161226 remaining. 

45.00% completed. 0:00:05.443253  elapsed. 0:00:06.652865 remaining. 

50.00% completed. 0:00:06.585315  elapsed. 0:00:06.585315 remaining. 

55.00% completed. 0:00:06.587383  elapsed. 0:00:05.389677 remaining. 

60.00% completed. 0:00:07.664297  elapsed. 0:00:05.109531 remaining. 

65.00% completed. 0:00:07.667260  elapsed. 0:00:04.128525 remaining. 

70.00% completed. 0:00:08.7

The parameter `blt_order` could not be identified for input_uv  so the ordering cannot be restored.The output UVData object will have (time, baseline) ordering.
Recalculating uvw_array without adjusting visibility phases -- this can introduce significant errors if used incorrectly.


ValueError: Phase center id 1 is does not have an entry in `phase_center_catalog`, which has keys dict_keys([0]). All values in `phase_center_id_array` must be keys in `phase_center_catalog`. 

In [ ]:
#catalog_formatted.spectral_type = "spectral_index"
#catalog_formatted.spectral_index = np.zeros(cat.Ncomponents)
#catalog_formatted.reference_frequency = np.mean(uv.freq_array)#*astropy.units.Hz

In [17]:
print(output_uv.phase_center_id_array)
print(uv.phase_center_catalog)

[1 1 1 1 1 1 1 1 1 1]
{1: {'cat_name': 'zenith_at_jd2460839.792738', 'cat_type': 'sidereal', 'cat_lon': 4.340978118544788, 'cat_lat': 0.6508804364976628, 'cat_frame': 'icrs', 'cat_epoch': 2000.0, 'cat_times': None, 'cat_pm_ra': None, 'cat_pm_dec': None, 'cat_vrad': None, 'cat_dist': None, 'info_source': 'user'}}


In [ ]:
uv.phase_center_app_ra = uv.phase_center_app_ra[:uv.Nblts]
uv.phase_center_app_dec = uv.phase_center_app_dec[:uv.Nblts]
uv.phase_center_frame_pa = uv.phase_center_frame_pa[:uv.Nblts]
uv.phase_center_id_array = uv.phase_center_id_array[:uv.Nblts]

In [18]:
output_uv.phase_to_time(np.mean(output_uv.time_array))

In [19]:
output_uv.check()

True

In [20]:
output_uv.data_array

array([[[0.05363051-0.00871819j],
        [0.05350005-0.00948594j]],

       [[0.05364847-0.00859533j],
        [0.05351977-0.00936334j]],

       [[0.05366613-0.00847247j],
        [0.0535392 -0.00924074j]],

       [[0.05368353-0.00834953j],
        [0.05355835-0.00911806j]],

       [[0.05370063-0.0082266j ],
        [0.05357721-0.00899538j]],

       [[0.05371746-0.00810359j],
        [0.05359581-0.00887262j]],

       [[0.05373401-0.00798054j],
        [0.05361412-0.00874982j]],

       [[0.05375027-0.00785751j],
        [0.05363214-0.00862703j]],

       [[0.05376625-0.00773439j],
        [0.05364989-0.00850415j]],

       [[0.05378195-0.0076113j ],
        [0.05366735-0.00838128j]]])